### Using Python pandas DataFrames to perform the queries in `sakila-sample-queries.sql`.

We use SQLAlchemy to connect to a sqlite version of the sakila database found in MySQL.  We only use SQLAlchemy to read the tables from the sakila database into pandas DataFrames.  From this point, we use pandas functionality to perform the requested queries and joins.

SQLAlchemy provides an Object Relational Mapper (ORM) in addtion to SQLAlchemy Core.  The ORM gives us an object oriented API to databases and Core is a standardized SQL Expression language to work with databases.  I will provide examples of using the ORM to perform the same requests in another Jupyter notebook.

In [1]:
# import the dependencies
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# create a connection to the database
engine = create_engine('sqlite:///../sqlite-sakila.db')

In [3]:
# list the available tables in the database
engine.table_names()

['actor',
 'address',
 'category',
 'city',
 'country',
 'customer',
 'film',
 'film_actor',
 'film_category',
 'film_text',
 'inventory',
 'language',
 'payment',
 'rental',
 'sqlite_sequence',
 'staff',
 'store']

In [4]:
# load the actor table into a pandas DataFrame
sql_query = """
select * from actor
"""
actor_df = pd.read_sql_query(sql_query, engine)
#actor_df.set_index('actor_id', inplace=True, )
actor_df.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2019-04-11 18:11:48
1,2,NICK,WAHLBERG,2019-04-11 18:11:48
2,3,ED,CHASE,2019-04-11 18:11:48
3,4,JENNIFER,DAVIS,2019-04-11 18:11:48
4,5,JOHNNY,LOLLOBRIGIDA,2019-04-11 18:11:48


In [5]:
# 1a. Display the first and last names of all actors from the table actor.
# this also works: actor_df.loc[:, ['first_name', 'last_name']]
actor_df[['first_name', 'last_name']].head()

,first_name,last_name
0,PENELOPE,GUINESS
1,NICK,WAHLBERG
2,ED,CHASE
3,JENNIFER,DAVIS
4,JOHNNY,LOLLOBRIGIDA


In [6]:
# 1b.Display the first and last name of each actor in a single column in upper case letters. Name the column Actor Name
first_name = actor_df.first_name.map(lambda x: x.upper())
last_name = actor_df.last_name.map(lambda x: x.upper())
actor_df['Actor Name'] = first_name + ' ' + last_name
actor_df['Actor Name'].head()

0       PENELOPE GUINESS
1          NICK WAHLBERG
2               ED CHASE
3         JENNIFER DAVIS
4    JOHNNY LOLLOBRIGIDA
Name: Actor Name, dtype: object

In [7]:
# 2a. You need to find the ID number, first name, and last name of an actor, of whom you know only the first name, "Joe."
actor_df[['actor_id', 'first_name', 'last_name']][actor_df['first_name'] == 'JOE']

,actor_id,first_name,last_name
8,9,JOE,SWANK


In [8]:
# 2b.  Find all actors whose last name contain the letters GEN
actor_df[['actor_id', 'first_name', 'last_name']][actor_df['last_name'].str.contains('GEN', regex=False)]

,actor_id,first_name,last_name
13,14,VIVIEN,BERGEN
40,41,JODIE,DEGENERES
106,107,GINA,DEGENERES
165,166,NICK,DEGENERES


In [9]:
# 2c. Find all actors whose last names contain the letters LI. This time, order the rows by last name and first name, in that order:
actor_df[['actor_id', 'first_name', 'last_name']][actor_df['last_name'].str.contains('LI', regex=False)].sort_values(by=['last_name', 'first_name'])

,actor_id,first_name,last_name
85,86,GREG,CHAPLIN
81,82,WOODY,JOLIE
33,34,AUDREY,OLIVIER
14,15,CUBA,OLIVIER
171,172,GROUCHO,WILLIAMS
136,137,MORGAN,WILLIAMS
71,72,SEAN,WILLIAMS
82,83,BEN,WILLIS
95,96,GENE,WILLIS
163,164,HUMPHREY,WILLIS


In [10]:
# 2d. Using IN, display the country_id and country columns of the following countries: Afghanistan, Bangladesh, and China:
sql_query = """
SELECT * FROM country
"""
country_df = pd.read_sql_query(sql_query, engine)
#country_df.set_index('country_id', inplace=True, )
country_df[['country_id', 'country']][country_df['country'].isin(['Afghanistan', 'Bangladesh', 'China'])]

,country_id,country
0,1,Afghanistan
11,12,Bangladesh
22,23,China


In [11]:
# 3a. You want to keep a description of each actor...
# In pandas - straight forward to add a column and we are not limited by type
actor_df['description'] = 'To be added later...'
actor_df.head()

,actor_id,first_name,last_name,last_update,Actor Name,description
0,1,PENELOPE,GUINESS,2019-04-11 18:11:48,PENELOPE GUINESS,To be added later...
1,2,NICK,WAHLBERG,2019-04-11 18:11:48,NICK WAHLBERG,To be added later...
2,3,ED,CHASE,2019-04-11 18:11:48,ED CHASE,To be added later...
3,4,JENNIFER,DAVIS,2019-04-11 18:11:48,JENNIFER DAVIS,To be added later...
4,5,JOHNNY,LOLLOBRIGIDA,2019-04-11 18:11:48,JOHNNY LOLLOBRIGIDA,To be added later...


In [12]:
# 3b. Drop the description column
actor_df.drop('description', axis=1, inplace=True)
actor_df.head()

,actor_id,first_name,last_name,last_update,Actor Name
0,1,PENELOPE,GUINESS,2019-04-11 18:11:48,PENELOPE GUINESS
1,2,NICK,WAHLBERG,2019-04-11 18:11:48,NICK WAHLBERG
2,3,ED,CHASE,2019-04-11 18:11:48,ED CHASE
3,4,JENNIFER,DAVIS,2019-04-11 18:11:48,JENNIFER DAVIS
4,5,JOHNNY,LOLLOBRIGIDA,2019-04-11 18:11:48,JOHNNY LOLLOBRIGIDA


In [13]:
# 4a. List the last names of actors, as well as how many actors have that last name.
actor_df['last_name'].groupby(actor_df['last_name']).count()

last_name
AKROYD         3
ALLEN          3
ASTAIRE        1
BACALL         1
BAILEY         2
BALE           1
BALL           1
BARRYMORE      1
BASINGER       1
BENING         2
BERGEN         1
BERGMAN        1
BERRY          3
BIRCH          1
BLOOM          1
BOLGER         2
BRIDGES        1
BRODY          2
BULLOCK        1
CAGE           2
CARREY         1
CHAPLIN        1
CHASE          2
CLOSE          1
COSTNER        1
CRAWFORD       2
CRONYN         2
CROWE          1
CRUISE         1
CRUZ           1
              ..
POSEY          1
PRESLEY        1
REYNOLDS       1
RYDER          1
SILVERSTONE    2
SINATRA        1
SOBIESKI       1
STALLONE       1
STREEP         2
SUVARI         1
SWANK          1
TANDY          2
TAUTOU         1
TEMPLE         4
TOMEI          1
TORN           3
TRACY          2
VOIGHT         1
WAHLBERG       2
WALKEN         1
WAYNE          1
WEST           2
WILLIAMS       3
WILLIS         3
WILSON         1
WINSLET        2
WITHERSPOON    1
WOOD

In [14]:
# 4b.   List last names of actors and the number of actors who have that last name, but only for names that are shared by at least two actors.
grouped_actor = actor_df['last_name'].groupby(actor_df['last_name']).count()
grouped_actor[grouped_actor > 1]

last_name
AKROYD         3
ALLEN          3
BAILEY         2
BENING         2
BERRY          3
BOLGER         2
BRODY          2
CAGE           2
CHASE          2
CRAWFORD       2
CRONYN         2
DAVIS          3
DEAN           2
DEE            2
DEGENERES      3
DENCH          2
DEPP           2
DUKAKIS        2
FAWCETT        2
GARLAND        3
GOODING        2
GUINESS        3
HACKMAN        2
HARRIS         3
HOFFMAN        3
HOPKINS        3
HOPPER         2
JACKMAN        2
JOHANSSON      3
KEITEL         3
KILMER         5
MCCONAUGHEY    2
MCKELLEN       2
MCQUEEN        2
MONROE         2
MOSTEL         2
NEESON         2
NOLTE          4
OLIVIER        2
PALTROW        2
PECK           3
PENN           2
SILVERSTONE    2
STREEP         2
TANDY          2
TEMPLE         4
TORN           3
TRACY          2
WAHLBERG       2
WEST           2
WILLIAMS       3
WILLIS         3
WINSLET        2
WOOD           2
ZELLWEGER      3
Name: last_name, dtype: int64

In [15]:
# 4c. The actor HARPO WILLIAMS was accidentally entered in the actor table as GROUCHO WILLIAMS. Write a query to fix the record.
# look-up 'Groucho Williams'

print(actor_df[(actor_df['first_name'] == "GROUCHO") & (actor_df['last_name'] ==  "WILLIAMS")])
actor_df.loc[171, 'first_name'] =  "HARPO"
print(actor_df[(actor_df['first_name'] == "HARPO") & (actor_df['last_name'] ==  "WILLIAMS")])

     actor_id first_name last_name          last_update        Actor Name
171       172    GROUCHO  WILLIAMS  2019-04-11 18:11:48  GROUCHO WILLIAMS
     actor_id first_name last_name          last_update        Actor Name
171       172      HARPO  WILLIAMS  2019-04-11 18:11:48  GROUCHO WILLIAMS


In [16]:
# 4d.  Undo the changes you just made to the df.
print(actor_df[(actor_df['first_name'] == "HARPO") & (actor_df['last_name'] ==  "WILLIAMS")])
actor_df.loc[171, 'first_name'] =  "GROUCHO"
print(actor_df[(actor_df['first_name'] == "GROUCHO") & (actor_df['last_name'] ==  "WILLIAMS")])

     actor_id first_name last_name          last_update        Actor Name
171       172      HARPO  WILLIAMS  2019-04-11 18:11:48  GROUCHO WILLIAMS
     actor_id first_name last_name          last_update        Actor Name
171       172    GROUCHO  WILLIAMS  2019-04-11 18:11:48  GROUCHO WILLIAMS


In [17]:
# 6a.  Use JOIN to display the first and last names, as well as the address, of each staff member. Use the tables staff and address:

# read the address and staff tables into corresponding pandas DataFrames.
sql_query = """
SELECT * FROM staff
"""
staff_df = pd.read_sql_query(sql_query, engine)

sql_query = """
SELECT * FROM address
"""
address_df = pd.read_sql_query(sql_query, engine)

combined_df = pd.merge(staff_df, address_df, on='address_id')
combined_df[['first_name', 'last_name', 'address']]

,first_name,last_name,address
0,Mike,Hillyer,23 Workhaven Lane
1,Jon,Stephens,1411 Lillydale Drive


In [18]:
# 6b. Use JOIN to display the total amount rung up by each staff member in August of 2005. Use tables staff and payment

# read the payment table into a pandas DataFrame
sql_query = """
SELECT * FROM payment
"""
payment_df = pd.read_sql_query(sql_query, engine)

# join the dataframes on staff_id
staff_payment_df = pd.merge(staff_df, payment_df, on='staff_id')

staff_payment_df.loc[0,'payment_date']

'2005-05-25 11:30:37.000'

In [19]:
import datetime
from datetime import datetime

# write a specific function to make a datetime object from the date string
# in the payment_df.  We are using the knowledge of the format provided.
def convert_date_from_string(string):
    return datetime.strptime(string, '%Y-%m-%d %H:%M:%S.%f')

# now add a column to staff_payment_df with the new datetime object
staff_payment_df['payment_datetime'] = staff_payment_df['payment_date'].apply(convert_date_from_string)
staff_payment_df['payment_datetime'].head()

0   2005-05-25 11:30:37
1   2005-05-28 10:35:23
2   2005-06-15 00:54:12
3   2005-06-16 15:18:57
4   2005-06-18 08:41:48
Name: payment_datetime, dtype: datetime64[ns]

In [20]:
# filter the DataFrame based on the month = 8, year = 2005
staff_payment_subdf = staff_payment_df[(staff_payment_df['payment_datetime'].map(lambda x: x.month) == 8)\
                                       & (staff_payment_df['payment_datetime'].map(lambda x: x.year) == 2005)]

# now group by staff_id and aggregate by the sum
grouped_staff_payment_subdf = staff_payment_subdf.groupby('staff_id')
grouped_payment = grouped_staff_payment_subdf['amount']
grouped_payment.agg('sum')

staff_id
1    11853.65
2    12218.48
Name: amount, dtype: float64

In [21]:
# 6c. List each film and the number of actors who are listed for that film. Use tables film_actor and film. Use inner join.

# Load the two tables into pandas DataFrames
sql_query = """
SELECT * FROM film_actor
"""
film_actor_df = pd.read_sql_query(sql_query, engine)

sql_query = """
SELECT * FROM film
"""
film_df = pd.read_sql_query(sql_query, engine)

# merge the two DataFrames on film_id
combined_df = pd.merge(film_actor_df, film_df, on='film_id')
combined_df.head()

,actor_id,film_id,last_update_x,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_y
0,1,1,2019-04-11 18:11:48,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2019-04-11 18:11:48
1,10,1,2019-04-11 18:11:48,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2019-04-11 18:11:48
2,20,1,2019-04-11 18:11:48,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2019-04-11 18:11:48
3,30,1,2019-04-11 18:11:48,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2019-04-11 18:11:48
4,40,1,2019-04-11 18:11:48,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2019-04-11 18:11:48


In [22]:
grouped_film = combined_df['title'].groupby(combined_df['title']).count()
grouped_film

title
ACADEMY DINOSAUR        10
ACE GOLDFINGER           4
ADAPTATION HOLES         5
AFFAIR PREJUDICE         5
AFRICAN EGG              5
AGENT TRUMAN             7
AIRPLANE SIERRA          5
AIRPORT POLLOCK          4
ALABAMA DEVIL            9
ALADDIN CALENDAR         8
ALAMO VIDEOTAPE          4
ALASKA PHANTOM           7
ALI FOREVER              5
ALICE FANTASIA           4
ALIEN CENTER             6
ALLEY EVOLUTION          5
ALONE TRIP               8
ALTER VICTORY            4
AMADEUS HOLY             6
AMELIE HELLFIGHTERS      6
AMERICAN CIRCUS          5
AMISTAD MIDSUMMER        4
ANACONDA CONFESSIONS     5
ANALYZE HOOSIERS         5
ANGELS LIFE              9
ANNIE IDENTITY           3
ANONYMOUS HUMAN          9
ANTHEM LUKE              2
ANTITRUST TOMATOES       7
ANYTHING SAVANNAH        3
                        ..
WHALE BIKINI             5
WHISPERER GIANT          3
WIFE TURN                6
WILD APOLLO              4
WILLOW TRACY             2
WIND PHANTOM          

In [25]:
#6d.  How many copies of the film Hunchback Impossible exist in the inventory system?
sql_query = """
SELECT * FROM inventory
"""
inventory_df = pd.read_sql_query(sql_query, engine)
inventory_df.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2019-04-11 18:11:48
1,2,1,1,2019-04-11 18:11:48
2,3,1,1,2019-04-11 18:11:48
3,4,1,1,2019-04-11 18:11:48
4,5,1,2,2019-04-11 18:11:48


In [26]:
# Basically perform a subquery by first finding the film_id of 'Hunchback Impossible'
target_film_id = film_df[film_df['title'] == 'HUNCHBACK IMPOSSIBLE'].film_id
print(target_film_id)

438    439
Name: film_id, dtype: int64


In [27]:
# Now select all films with this film_id in the inventory_df
target_film = inventory_df[inventory_df['film_id'] == 439]
len(target_film)

6

In [28]:
# 6e.  Using the tables payment and customer and the JOIN command, list the total paid by each customer. List the customers alphabetically by last name:

# first read in the customer table
sql_query = """
SELECT * FROM customer
"""
customer_df = pd.read_sql_query(sql_query, engine)
customer_df.head()

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36.000,2019-04-11 18:11:49
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36.000,2019-04-11 18:11:49
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36.000,2019-04-11 18:11:49
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36.000,2019-04-11 18:11:49
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36.000,2019-04-11 18:11:49


In [29]:
# join the dataframes on customer_id
customer_payment_df = pd.merge(payment_df, customer_df, on='customer_id')

In [30]:
# A function to perform summary computations after grouping a dataframe

def group_and_summarize_dataframe (df, grouping, stats):
    """Compute summary quantities for a pandas dataframe after grouping.
    
    Args:
        df: A pandas dataframe
        grouping: A column name or a list of column names to group
            the dataframe with.
        stats: A list of tuples. Each tuple is of the form:
            
            (name, column, function)
        
        where `name` is the desired output name for the summary quantity,
        `column` is the target column of the data frame, and `function` is 
        the function used to compute the summary.
        
    Returns:
        A pandas dataframe with the output summaries.
        
    Raises:
        None.  At this time, there is no error checking for the input parameters.
    """
        
    # Create an empty dataframe to store the results.
    results_df = pd.DataFrame()
    
    grouped_df = df.groupby(grouping)
    
    for name, column, agg_func in stats:
        results_df[name] = grouped_df[column].agg(agg_func)
        
    return results_df

In [31]:
total_payments_df = group_and_summarize_dataframe(customer_payment_df, 'customer_id',\
                                           [('first_name', 'first_name', lambda x: x.unique()),\
                                            ('last_name', 'last_name', lambda x: x.unique()),\
                                            ('Total Amount Paid', 'amount', 'sum')])
                                                
total_payments_df.sort_values(by=['last_name', 'first_name'])

,first_name,last_name,Total Amount Paid
customer_id,,,
505,RAFAEL,ABNEY,97.79
504,NATHANIEL,ADAM,133.72
36,KATHLEEN,ADAMS,92.73
96,DIANA,ALEXANDER,105.73
470,GORDON,ALLARD,160.68
27,SHIRLEY,ALLEN,126.69
220,CHARLENE,ALVAREZ,114.73
11,LISA,ANDERSON,106.76
326,JOSE,ANDREW,96.75


In [32]:
# 7a. The music of Queen and Kris Kristofferson have seen an unlikely resurgence. As an unintended consequence, 
# films starting with the letters K and Q have also soared in popularity. Use subqueries to display the titles of 
# movies starting with the letters K and Q whose language is English.

# First import the needed tables as DataFrames
sql_query = """
SELECT * FROM language
"""
language_df = pd.read_sql_query(sql_query, engine)
language_df.head()

,language_id,name,last_update
0,1,English,2019-04-11 18:11:48
1,2,Italian,2019-04-11 18:11:48
2,3,Japanese,2019-04-11 18:11:48
3,4,Mandarin,2019-04-11 18:11:48
4,5,French,2019-04-11 18:11:48


In [33]:
# Now find the language_id of films in English
language_df[language_df['name'] == 'English']

,language_id,name,last_update
0,1,English,2019-04-11 18:11:48


In [34]:
# Now pick out all English films and then subset based on starts with 'K' or 'Q'
english_films = film_df[film_df['language_id'] == 1]

In [35]:

def starts_with_values(string, values):
    return any([string.startswith(val) for val in values])

english_films[english_films['title'].map(lambda x: starts_with_values(x, ['K', 'Q']))].title

492       KANE EXORCIST
493         KARATE MOON
494    KENTUCKIAN GIANT
495       KICK SAVANNAH
496    KILL BROTHERHOOD
497     KILLER INNOCENT
498      KING EVOLUTION
499          KISS GLORY
500       KISSING DOLLS
501       KNOCK WARLOCK
502    KRAMER CHOCOLATE
503       KWAI HOMEWARD
705          QUEEN LUKE
706     QUEST MUSSOLINI
707         QUILLS BULL
Name: title, dtype: object

In [36]:
# 7b. Use subqueries to display all actors who appear in the film Alone Trip.
# we already have the film_actor, film, and actor tables in the corresponding DataFrames from above.

# Find the film_id of the film 'Alone Trip'
film_df[film_df['title'] == 'ALONE TRIP'].film_id

16    17
Name: film_id, dtype: int64

In [37]:
actors_in_alonetrip = film_actor_df[film_actor_df['film_id'] == 17].actor_id

In [38]:
actor_df[actor_df['actor_id'].isin(actors_in_alonetrip)]

,actor_id,first_name,last_name,last_update,Actor Name
2,3,ED,CHASE,2019-04-11 18:11:48,ED CHASE
11,12,KARL,BERRY,2019-04-11 18:11:48,KARL BERRY
12,13,UMA,WOOD,2019-04-11 18:11:48,UMA WOOD
81,82,WOODY,JOLIE,2019-04-11 18:11:48,WOODY JOLIE
99,100,SPENCER,DEPP,2019-04-11 18:11:48,SPENCER DEPP
159,160,CHRIS,DEPP,2019-04-11 18:11:48,CHRIS DEPP
166,167,LAURENCE,BULLOCK,2019-04-11 18:11:48,LAURENCE BULLOCK
186,187,RENEE,BALL,2019-04-11 18:11:48,RENEE BALL


In [39]:
# 7c. You want to run an email marketing campaign in Canada, for which you will need the names and email addresses of all Canadian customers. Use joins to retrieve this information.
# Need to get the city table in 
sql_query = """
SELECT * FROM city
"""
city_df = pd.read_sql_query(sql_query, engine)
city_df.head()

,city_id,city,country_id,last_update
0,1,A Corua (La Corua),87,2019-04-11 18:11:48
1,2,Abha,82,2019-04-11 18:11:48
2,3,Abu Dhabi,101,2019-04-11 18:11:48
3,4,Acua,60,2019-04-11 18:11:48
4,5,Adana,97,2019-04-11 18:11:48


In [40]:
# Now use a series of joins - could also use a sequence of sub_queries
address_joined = pd.merge(customer_df,\
                          pd.merge(address_df,\
                                   pd.merge(city_df, country_df, on='country_id'),\
                                  on='city_id'),\
                         on='address_id')
address_joined[['first_name', 'last_name', 'email']][address_joined['country']=='Canada']

,first_name,last_name,email
188,LORETTA,CARPENTER,LORETTA.CARPENTER@sakilacustomer.org
409,CURTIS,IRBY,CURTIS.IRBY@sakilacustomer.org
435,TROY,QUIGLEY,TROY.QUIGLEY@sakilacustomer.org
462,DARRELL,POWER,DARRELL.POWER@sakilacustomer.org
475,DERRICK,BOURQUE,DERRICK.BOURQUE@sakilacustomer.org


In [41]:
# 7d. Sales have been lagging among young families, and you wish to target all family movies for a promotion. Identify all movies categorized as family films.

# Need to load the film_category and category tables

sql_query = """
SELECT * FROM film_category
"""
film_category_df = pd.read_sql_query(sql_query, engine)

sql_query = """
SELECT * FROM category
"""
category_df = pd.read_sql_query(sql_query, engine)


In [42]:
# We can use sub_queries:
family_ids = category_df[category_df['name'] == 'Family'].category_id
film_ids = film_category_df[film_category_df['category_id'].isin(family_ids)].film_id
family_films = film_df[film_df['film_id'].isin(film_ids)]
family_films[['title']]

,title
4,AFRICAN EGG
30,APACHE DIVINE
42,ATLANTIS CAUSE
49,BAKED CLEOPATRA
52,BANG KWAI
62,BEDAZZLED MARRIED
70,BILKO ANONYMOUS
79,BLANKET BEVERLY
81,BLOOD ARGONAUTS
82,BLUES INSTINCT


In [43]:
# We could also have done this joins
film_category_join_df = pd.merge(film_df,\
                                pd.merge(film_category_df, category_df, on='category_id'),
                                 on='film_id')
film_category_join_df[film_category_join_df['name']=="Family"].title

4                 AFRICAN EGG
30              APACHE DIVINE
42             ATLANTIS CAUSE
49            BAKED CLEOPATRA
52                  BANG KWAI
62          BEDAZZLED MARRIED
70            BILKO ANONYMOUS
79            BLANKET BEVERLY
81            BLOOD ARGONAUTS
82             BLUES INSTINCT
93           BRAVEHEART HUMAN
138             CHASING FIGHT
144           CHISUM BEHAVIOR
146            CHOCOLAT HARRY
174          CONFUSED CANDLES
182     CONVERSATION DOWNHILL
212                DATE SPEED
230        DINOSAUR SECRETARY
261                DUMBO LUST
268          EARRING INSTINCT
272          EFFECT GLADIATOR
308              FEUD FROGMEN
314          FINDING ANACONDA
344         GABLES METROPOLIS
347               GANDHI KWAI
358        GLADIATOR WESTWARD
376              GREASE YOUTH
390             HALF OUTFIELD
418               HOCUS FRIDA
427            HOMICIDE PEACH
                ...          
549            MAGUIRE APACHE
556        MANCHURIAN CURTAIN
602       

In [44]:
# 7e. Display the most frequently rented movies in descending order.
# Need to load the rental table

sql_query = """
SELECT * FROM rental"""
rental_df = pd.read_sql_query(sql_query, engine)
rental_df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30.000,367,130,2005-05-26 22:04:30.000,1,2019-04-11 18:11:49
1,2,2005-05-24 22:54:33.000,1525,459,2005-05-28 19:40:33.000,1,2019-04-11 18:11:49
2,3,2005-05-24 23:03:39.000,1711,408,2005-06-01 22:12:39.000,1,2019-04-11 18:11:49
3,4,2005-05-24 23:04:41.000,2452,333,2005-06-03 01:43:41.000,2,2019-04-11 18:11:49
4,5,2005-05-24 23:05:21.000,2079,222,2005-06-02 04:33:21.000,1,2019-04-11 18:11:49


In [45]:
# Now join film with (inventory joined with rental on inventory_id) on film_id

rental_joined_df = pd.merge(film_df, pd.merge(inventory_df, rental_df, on='inventory_id'), on='film_id')
rental_joined_df.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,...,last_update,inventory_id,store_id,last_update_x,rental_id,rental_date,customer_id,return_date,staff_id,last_update_y
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2019-04-11 18:11:48,1,1,2019-04-11 18:11:48,4863,2005-07-08 19:03:15.000,431,2005-07-11 21:29:15.000,2,2019-04-11 18:11:49
1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2019-04-11 18:11:48,1,1,2019-04-11 18:11:48,11433,2005-08-02 20:13:10.000,518,2005-08-11 21:35:10.000,1,2019-04-11 18:11:49
2,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2019-04-11 18:11:48,1,1,2019-04-11 18:11:48,14714,2005-08-21 21:27:43.000,279,2005-08-30 22:26:43.000,1,2019-04-11 18:11:50
3,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2019-04-11 18:11:48,2,1,2019-04-11 18:11:48,972,2005-05-30 20:21:07.000,411,2005-06-06 00:36:07.000,1,2019-04-11 18:11:49
4,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,...,2019-04-11 18:11:48,2,1,2019-04-11 18:11:48,2117,2005-06-17 20:24:00.000,170,2005-06-23 17:45:00.000,2,2019-04-11 18:11:49


In [46]:
# use my group_and_summarize function to get the desired output
total_rentals_df = group_and_summarize_dataframe(rental_joined_df, 'film_id',\
                                           [('film_id', 'film_id', lambda x: x.unique()),\
                                            ('title', 'title', lambda x: x.unique()),\
                                            ('Frequency', 'film_id', 'count')])
                                                
total_rentals_df.sort_values(by=['Frequency'], ascending=False)

,film_id,title,Frequency
film_id,,,
103,103,BUCKET BROTHERHOOD,34
738,738,ROCKETEER MOTHER,33
730,730,RIDGEMONT SUBMARINE,32
382,382,GRIT CLOCKWORK,32
489,489,JUGGLER HARDLY,32
331,331,FORWARD TEMPLE,32
767,767,SCALAWAG DUCK,32
1000,1000,ZORRO ARK,31
891,891,TIMBERLAND SKY,31


In [50]:
# 7f.  Write a query to display how much business, in dollars, each store brought in.

# First read in the store table to a DataFrame
sql_query = """
SELECT * FROM store
"""
store_df = pd.read_sql_query(sql_query, engine)
store_df

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2019-04-11 18:11:48
1,2,2,2,2019-04-11 18:11:48


In [51]:
# For sanity check.  What is the total amount of rentals
sum(pd.merge(payment_df, rental_df, on='rental_id').amount)

67406.55999999207

In [52]:
# To get the total amount from each store we need to merge and the store_id comes from the customer - not the rental table
store_amount_df = pd.merge(store_df,\
                          pd.merge(customer_df,\
                                  pd.merge(payment_df, rental_df, on='rental_id'),\
                                  left_on='customer_id', right_on='customer_id_x'),
                          on='store_id')
store_amount_df.head()

,store_id,manager_staff_id,address_id_x,last_update_x,customer_id,first_name,last_name,email,address_id_y,active,...,rental_id,amount,payment_date,last_update_x,rental_date,inventory_id,customer_id_y,return_date,staff_id_y,last_update_y
0,1,1,1,2019-04-11 18:11:48,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,...,76.0,2.99,2005-05-25 11:30:37.000,2019-04-11 18:11:50,2005-05-25 11:30:37.000,3021,1,2005-06-03 12:00:37.000,2,2019-04-11 18:11:49
1,1,1,1,2019-04-11 18:11:48,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,...,573.0,0.99,2005-05-28 10:35:23.000,2019-04-11 18:11:50,2005-05-28 10:35:23.000,4020,1,2005-06-03 06:32:23.000,1,2019-04-11 18:11:49
2,1,1,1,2019-04-11 18:11:48,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,...,1185.0,5.99,2005-06-15 00:54:12.000,2019-04-11 18:11:50,2005-06-15 00:54:12.000,2785,1,2005-06-23 02:42:12.000,2,2019-04-11 18:11:49
3,1,1,1,2019-04-11 18:11:48,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,...,1422.0,0.99,2005-06-15 18:02:53.000,2019-04-11 18:11:50,2005-06-15 18:02:53.000,1021,1,2005-06-19 15:54:53.000,2,2019-04-11 18:11:49
4,1,1,1,2019-04-11 18:11:48,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,...,1476.0,9.99,2005-06-15 21:08:46.000,2019-04-11 18:11:50,2005-06-15 21:08:46.000,1407,1,2005-06-25 02:26:46.000,1,2019-04-11 18:11:49


In [53]:
group_and_summarize_dataframe(store_amount_df, 'store_id',\
                              [('store_id', 'store_id', lambda x: x.unique()),\
                               ('Total Amount', 'amount', 'sum')])


,store_id,Total Amount
store_id,,
1,1,36997.53
2,2,30409.03


In [54]:
# 7g.  Write a query to display for each store its store ID, city, and country.

store_address_df = pd.merge(store_df, pd.merge(address_df, pd.merge(city_df, country_df, on='country_id'),\
                                           on='city_id'),\
                           on='address_id')
store_address_df[['store_id', 'city', 'country']]

,store_id,city,country
0,1,Lethbridge,Canada
1,2,Woodridge,Australia


In [55]:
# 7h
# List the top five genres in gross revenue in descending order. (Hint: you may need to use the following tables: category, film_category, inventory, payment, and rental.)

genres_df = pd.merge(payment_df,\
                     pd.merge(rental_df,\
                              pd.merge(inventory_df,\
                                       pd.merge(film_category_df, category_df, on='category_id'),\
                                      on='film_id'),\
                             on='inventory_id'),\
                     on='rental_id')

revenue_df = group_and_summarize_dataframe(genres_df, 'category_id',\
                                           [('name', 'name', (lambda x: x.unique())),\
                                            ('category_id', 'category_id', (lambda x: x.unique())),\
                                            ('Gross Revenue', 'amount', 'sum')])
revenue_df.sort_values(by='Gross Revenue',ascending=False)[:5]

,name,category_id,Gross Revenue
category_id,,,
15,Sports,15,5314.21
14,Sci-Fi,14,4756.98
2,Animation,2,4656.30
7,Drama,7,4587.39
5,Comedy,5,4383.58
